Omitted these sections from the slides - they just set up components already defined in the `Crop Rotation Tutorial` notebook.

In [1]:
using JuMP
using GLPKMathProgInterface

crop = ["A", "B", "C"]

function get_action(x, j, M, N)
    plant = [i for i in 1:N if j in I[i] && getvalue(x[i, j]) == 1]
    if length(plant) > 0
        return plant[1]
    else
        harvest = [
            crop[i] for i in 1:N for r in 1:(t[i] - 1)
            if mod(j - r - 1, M) + 1 in I[i]
            && getvalue(x[i, mod(j - r - 1, M) + 1]) == 1]
        if length(harvest) > 0
            return "-"
        end
    end
    return " "
end

function get_schedule(x, M, N)
    "|" * join([get_action(x, j, M, N) for j in 1:M], "|") * "|"
end

function get_production(x, M, N)
    [sum(getvalue(x[i, j]) for j in I[i]) for i in 1:N]
end

get_production (generic function with 1 method)

# Part 2: Assigning schedules to plot areas

Generate crop schedules $k \in K$ which produce $a_{ik}$ units of crop $i$ per unit area.
By assigning areas to schedules, we minimise land use required to satisfy the given demand.

\begin{alignat*}{2}
& \min \,\, \sum_{k \in K} \lambda_{k} \\
& \sum a_{ik} \lambda_{k} \ge d_{i}, \quad i = 1 \dots N \\
& \lambda_{k} \ge 0, \quad k \in K \\
\end{alignat*}

The subproblem searches for feasible crop schedules with the following objective:

\begin{alignat*}{2}
& z = \min \,\, 1 - \sum_{i = 1}^{N} \sum_{j \in I[i]} \pi_{i} x_{ij} \\
\end{alignat*}

where $\pi_i$ are the dual variables associated with the demand coverage constraints.

## Define problem data

In [2]:
# Crop rotation schedule data.
M = 12             # Number of periods considered in the 12 month rotation.
N = 3              # Number of crops.
G = [3]            # Set of green manuring crops.
F = [[1, 2], [3]]  # Set of crop families.
t = [5, 4, 2, 1]   # Production time for each crop.
I = [              # Valid planting times.
    1:4, 1:12,
    vcat(1:3, 7:12),
    1:12
]
n = N+1            # Fallow 'crop'.

# Demand data.
d = [2, 4, 1];

## Construct a base model for the subproblem

* We create a single model instance here and will re-use it with different objectives.
* JuMP re-optimises if possible, starts from scratch if required.

In [3]:
model = Model(solver=GLPKSolverMIP())
@variable(model, x[i in 1:N+1, j in I[i]], Bin)
# One crop at a time.
@constraint(model,
    [j in 1:M],
    sum(
        sum(
            x[i, mod(j - r - 1, M) + 1]
            for r in 0:t[i]-1
            if mod(j - r - 1, M) + 1 in I[i])
        for i in 1:N+1) <= 1);
# Crops of same family.
@constraint(model,
    [f in F, j in 1:M],
    sum(
        sum(
            x[i, mod(j - r - 1, M) + 1]
            for r in 0:t[i]
            if mod(j - r - 1, M) + 1 in I[i])
        for i in f) <= 1);
# Fallow & green manure.
@constraint(model,
    sum(
        sum(x[i, j] for j in I[i])
        for i in G) == 1)
@constraint(model, sum(x[N + 1, j] for j in 1:M) == 1);

## Define pricing problem as a callable function

* This function solves for a crop rotation schedule under the fixed set of constraints with given dual values.
* We'll use this to solve the column generation subproblem which calculates a new column and its reduced costs.

In [4]:
# Construct and solve a crop schedule feasibility model with
# the given objective values applied to a planting of each
# crop type. Return the resulting objective value and a
# representation of the schedule.
function price_crop_rotation(L)
    @objective(model, Min,
        1 - sum(
            L[i] * x[i, j]
            for i in 1:N
            for j in I[i]))
    solve(model)
    getobjectivevalue(model), get_production(x, M, N), get_schedule(x, M, N)
end

price_crop_rotation (generic function with 1 method)

## Generate initial columns

* Naively construct some columns, enough to satisfy demand.
* This is likely to be a poor solution, more schedules will be needed.

In [5]:
columns = [
    Dict(
        "production" => [1, 0, 1],
        "schedule" => "|1| | | | |F| | |3| | | |"),
    Dict(
        "production" => [0, 1, 1],
        "schedule" => "|2| | |F| | | | |3| | | |")
]

2-element Array{Dict{String,Any},1}:
 Dict("production"=>[1, 0, 1],"schedule"=>"|1| | | | |F| | |3| | | |")
 Dict("production"=>[0, 1, 1],"schedule"=>"|2| | |F| | | | |3| | | |")

## Construct the initial master problem with the available crop schedules.

* There are only two schedules in the initial model.

\begin{alignat*}{2}
& \min \,\, \sum_{k \in K} \lambda_{k} \\
& \sum a_{ik} \lambda_{k} \ge d_{i}, \quad i = 1 \dots N \\
& \lambda_{k} \ge 0, \quad k \in K \\
\end{alignat*}

In [6]:
master_model = Model(solver=GLPKSolverLP())
@variable(master_model, l[i in 1:length(columns)] >= 0)
@constraint(
    master_model, demand_covered[j in 1:length(d)],
    sum(columns[i]["production"][j] * l[i] for i in 1:length(columns)) >= d[j])
@objective(master_model, Min, sum(l))
master_model

Minimization problem with:
 * 3 linear constraints
 * 2 variables
Solver is GLPKMathProgInterface.GLPKInterfaceLP.GLPK

## Bookkeeping

* Capture the area variables along with the stored column data.

In [7]:
for i in 1:length(columns)
    columns[i]["area"] = l[i]
end

In [8]:
columns

2-element Array{Dict{String,Any},1}:
 Dict("production"=>[1, 0, 1],"area"=>l[1],"schedule"=>"|1| | | | |F| | |3| | | |")
 Dict("production"=>[0, 1, 1],"area"=>l[2],"schedule"=>"|2| | |F| | | | |3| | | |")

# Column Generation Loop

1. Solve the master problem
2. Extract the constraint dual values
3. Update and solve the subproblem
4. If an improving column is found, add to the master and repeat

## Solve the master problem

* Assigns areas to each plot.
* Production area is optimal *for our current set of schedules*.

In [9]:
solve(master_model)
println("Total Area = ", getobjectivevalue(master_model))
println()

for column in columns
    println(column["schedule"], "   AREA = ", getvalue(column["area"]))
end

Total Area = 6.0

|1| | | | |F| | |3| | | |   AREA = 2.0
|2| | |F| | | | |3| | | |   AREA = 4.0


## Solve the pricing problem to generate a new column.

\begin{alignat*}{2}
& z = \min \,\, 1 - \sum_{i = 1}^{N} \sum_{j \in I[i]} \pi_{i} x_{ij} \\
\end{alignat*}

* An extra schedule is added which produces more of crop 2.
* New variable in the master problem can satisfy higher demand in a single cycle.

In [10]:
objective, production, schedule = price_crop_rotation(getdual(demand_covered))
println(" Objective : ", objective)
println("Production : ", production)
println("  Schedule : ", schedule)

 Objective : -0.9999999999999998
Production : [0.0, 2.0, 1.0]
  Schedule : |-| |2|-|-|-|3|-| |2|-|-|


* Negative reduced costs indicate an improving column, so add it to the model.

## Create an additional variable.

* This modifies the master LP model in-memory so it can be reoptimised.

In [11]:
@variable(
    master_model, l_3 >= 0, objective=1.0,
    inconstraints=demand_covered, coefficients=production)
master_model

Minimization problem with:
 * 3 linear constraints
 * 3 variables
Solver is GLPKMathProgInterface.GLPKInterfaceLP.GLPK

## Update column bookkeeping

* Keep track of the new column data and master variable.

In [12]:
new_column = Dict(
    "production" => production,
    "schedule" => schedule,
    "area" => l_3)
push!(columns, new_column)
columns

3-element Array{Dict{String,Any},1}:
 Dict("production"=>[1, 0, 1],"area"=>l[1],"schedule"=>"|1| | | | |F| | |3| | | |")     
 Dict("production"=>[0, 1, 1],"area"=>l[2],"schedule"=>"|2| | |F| | | | |3| | | |")     
 Dict("production"=>[0.0, 2.0, 1.0],"area"=>l_3,"schedule"=>"|-| |2|-|-|-|3|-| |2|-|-|")

## Reoptimise the master problem

* Total required area is reduced with the updated set of schedules.

In [13]:
solve(master_model)
println("Total Area = ", getobjectivevalue(master_model))
println()

for column in columns
    println(column["schedule"], "   AREA = ", getvalue(column["area"]))
end

Total Area = 4.0

|1| | | | |F| | |3| | | |   AREA = 2.0
|2| | |F| | | | |3| | | |   AREA = 0.0
|-| |2|-|-|-|3|-| |2|-|-|   AREA = 2.0


## Final Loop

* Run column generation iterations using a `while` loop.

In [14]:
while true
    solve(master_model)
    objective, production, schedule = price_crop_rotation(getdual(demand_covered))
    if objective >= 0
        display("DONE")
        break
    end
    @variable(
        master_model, z >= 0, objective=0.0,
        inconstraints=demand_covered, coefficients=production)
    new_column = Dict("production" => production, "schedule" => schedule, "area" => z)
    push!(columns, new_column)
    display("COLUMN ADDED")
    display(new_column)
end

"COLUMN ADDED"

Dict{String,Any} with 3 entries:
  "production" => [1.0, 1.0, 1.0]
  "area"       => z
  "schedule"   => "|3|-|1|-|-|-|-| |2|-|-|-|"

"DONE"

## View the resulting schedules

In [15]:
for column in columns
    println(column["schedule"], "   AREA = ", getvalue(column["area"]))
end

|1| | | | |F| | |3| | | |   AREA = 0.0
|2| | |F| | | | |3| | | |   AREA = 0.0
|-| |2|-|-|-|3|-| |2|-|-|   AREA = 0.0
|3|-|1|-|-|-|-| |2|-|-|-|   AREA = 4.0
